In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from sklearn.svm import NuSVR
from sklearn.model_selection import train_test_split
import os
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor

def calculate_metrics(y_true, y_pred):    
    r2 = r2_score(y_true, y_pred)
    print(" R2 of the model is ", r2)
    rms = sqrt(mean_squared_error(y_true, y_pred))
    print(" RMSE of the model is ", rms)
    mae = mean_absolute_error(y_true, y_pred)
    print(" MAE of the model is ", mae)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(" MAPE of the model is ", mape)
    
def make_plot(y_test, y_pred):
    plt.figure(figsize=(15, 5))
    plt.plot(y_pred)
    plt.plot(y_test.values)
    plt.legend(['Predicted', 'Actual'])
    
def make_scatterplot(y_test, y_pred):
    plt.scatter(y_test, y_pred)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')


DATA_PATH = "./Simulations/"
datapoints = {}

for root, dirs, files in os.walk(DATA_PATH):
    if 'FormFactor.json' in files:
        simul_id = root[len(DATA_PATH):]
        
        f = open(root + '/FormFactor.json')
        data = json.load(f)
        form_factor = pd.DataFrame(data)[1]
        
        if 'thickness.json' in files:
            f = open(root + '/thickness.json')
            thickness = json.load(f)
        else:
            thickness = np.nan
            
        if 'apl.json' in files:
            f = open(root + '/apl.json')
            data = json.load(f)
            
            value_mean = np.mean(list(data.values()))
            
            if np.isnan(value_mean):
                apl = np.nan
                apl_len = np.nan
            else:
                apl = value_mean
                apl_len = len(list(data.values()))
        else:
            apl = np.nan
            apl_len = np.nan
            
        if 'TotalDensity.json' in files:
            f = open(root + '/TotalDensity.json')
            data = json.load(f)
            
            total_density = pd.DataFrame(data)
            total_density_len = total_density.shape[0]
        else:
            total_density = np.nan
            total_density_len = np.nan
            
        result = {
            'form_factor': form_factor,
            'thickness': thickness,
            'apl': apl,
            'apl_len': apl_len,
            'total_density': total_density,
            'total_density_len': total_density_len
        }
        
        datapoints[simul_id] = result

In [6]:
datapoints['28-0d5-d1d-0d5d1dcb43e775faf4e53c4f9ff255a67481bd38-9b487701b24d3fad83991e311188b08d3d5ea768'].keys()

dict_keys(['form_factor', 'thickness', 'apl', 'apl_len', 'total_density', 'total_density_len'])

In [14]:
datapoints['28-0d5-d1d-0d5d1dcb43e775faf4e53c4f9ff255a67481bd38-9b487701b24d3fad83991e311188b08d3d5ea768']['total_density'].shape

(180, 2)

In [7]:
len(datapoints.keys())

715

In [60]:
form_factor = []
thickness = []
apl = []

for key in datapoints:
    if not np.isnan(datapoints[key]['thickness']):
        form_factor.append(datapoints[key]['form_factor'])
        thickness.append(datapoints[key]['thickness'])

In [62]:
len(form_factor)

711

In [67]:
form_factor = np.array(form_factor)

In [68]:
form_factor.shape

(711, 1000)

In [70]:
df = pd.DataFrame(form_factor)
df['thickness'] = thickness

In [77]:
df_copy=df

In [78]:
x = df[['thickness']]

In [79]:
y = df.drop(['thickness'],axis=1)

In [81]:
y

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,6.344403,6.320138,6.247351,6.126077,5.956366,5.738296,5.471964,5.157489,4.795020,4.384730,...,0.680424,0.707091,0.735799,0.766320,0.798438,0.831960,0.866705,0.902513,0.939236,0.976743
1,22.490975,22.459825,22.366391,22.210718,21.992883,21.712994,21.371188,20.967635,20.502534,19.976114,...,4.380145,4.377129,4.372789,4.367126,4.360142,4.351838,4.342218,4.331287,4.319049,4.305511
2,13.783229,13.762646,13.700906,13.598035,13.454075,13.269087,13.043148,12.776350,12.468807,12.120644,...,2.249947,2.218988,2.187594,2.155785,2.123578,2.090991,2.058044,2.024753,1.991138,1.957217
3,21.447578,21.483182,21.589971,21.767875,22.016780,22.336524,22.726898,23.187651,23.718482,24.319047,...,2.408326,2.474208,2.538082,2.599919,2.659692,2.717376,2.772949,2.826390,2.877679,2.926800
4,13.997836,13.973720,13.901380,13.780851,13.612188,13.395470,13.130795,12.818287,12.458089,12.050368,...,0.242800,0.235561,0.228223,0.220795,0.213286,0.205709,0.198076,0.190400,0.182700,0.174993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,75.019056,74.984627,74.881354,74.709289,74.468513,74.159143,73.781326,73.335246,72.821115,72.239182,...,1.208284,1.180976,1.154276,1.128200,1.102765,1.077986,1.053877,1.030453,1.007729,0.985716
707,92.289343,92.246671,92.118676,91.905425,91.607026,91.223633,90.755444,90.202699,89.565682,88.844720,...,9.335528,9.133224,8.931639,8.730875,8.531033,8.332216,8.134524,7.938056,7.742911,7.549186
708,51.763699,51.740017,51.668979,51.550611,51.384957,51.172078,50.912051,50.604972,50.250954,49.850127,...,3.539732,3.460534,3.380503,3.299691,3.218150,3.135933,3.053098,2.969702,2.885806,2.801474
709,71.950184,71.915288,71.810614,71.636207,71.392140,71.078515,70.695464,70.243148,69.721756,69.131507,...,1.276403,1.347048,1.417474,1.487645,1.557526,1.627084,1.696287,1.765104,1.833502,1.901452


In [85]:
X_train, X_test, y_train, y_test = train_test_split(y, x, test_size=0.30, random_state=3)

In [89]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
683,30.243649,30.222959,30.160896,30.057484,29.912759,29.726775,29.499600,29.231315,28.922019,28.571825,...,1.418072,1.440353,1.461691,1.482089,1.501552,1.520083,1.537689,1.554375,1.570146,1.585010
211,15.608165,15.581932,15.503246,15.372140,15.188670,14.952918,14.664984,14.324994,13.933095,13.489456,...,1.020590,1.093957,1.167231,1.240335,1.313204,1.385779,1.458007,1.529842,1.601238,1.672155
16,38.608844,38.584945,38.513256,38.393803,38.226628,38.011789,37.749364,37.439444,37.082140,36.677577,...,2.717158,2.736302,2.755918,2.775984,2.796480,2.817386,2.838682,2.860347,2.882363,2.904711
8,44.793382,44.775384,44.721398,44.631443,44.505548,44.343760,44.146134,43.912738,43.643656,43.338979,...,0.017372,0.019730,0.022067,0.024383,0.026678,0.028951,0.031203,0.033433,0.035641,0.037828
685,38.550151,38.526036,38.453698,38.333164,38.164477,37.947699,37.682906,37.370196,37.009679,36.601487,...,0.530550,0.565889,0.602380,0.639994,0.678701,0.718469,0.759268,0.801067,0.843834,0.887537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,99.293898,99.264194,99.175093,99.026629,98.818860,98.551864,98.225745,97.840627,97.396658,96.894007,...,3.075976,3.078319,3.080949,3.083857,3.087032,3.090464,3.094143,3.098059,3.102201,3.106557
256,34.421472,34.397159,34.324230,34.202713,34.032658,33.814132,33.547223,33.232037,32.868701,32.457360,...,0.437434,0.439854,0.446857,0.458249,0.473722,0.492890,0.515329,0.540615,0.568342,0.598141
643,35.707526,35.691944,35.645199,35.567303,35.458274,35.318139,35.146931,34.944691,34.711466,34.447312,...,9.245041,9.192052,9.137834,9.082403,9.025778,8.967976,8.909018,8.848921,8.787705,8.725390
249,45.136668,45.104980,45.009929,44.851553,44.629915,44.345105,43.997236,43.586450,43.112909,42.576806,...,0.302417,0.312797,0.327396,0.345879,0.367843,0.392869,0.420561,0.450563,0.482565,0.516304


In [101]:
rf = RandomForestRegressor(n_jobs=-1, n_estimators=1000)

In [102]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

/tmp/ipykernel_153110/2739726120.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


In [103]:
calculate_metrics(y_test, y_pred)

 R2 of the model is  0.9031041890671557
 RMSE of the model is  0.09909941895718254
 MAE of the model is  0.05870610978489292
 MAPE of the model is  0.013415333853696787


In [109]:
nusvr = NuSVR()

In [110]:
nusvr.fit(X_train, y_train)
y_pred = nusvr.predict(X_test)

In [111]:
calculate_metrics(y_test, y_pred)

 R2 of the model is  0.9385095597413876
 RMSE of the model is  0.07894459635293546
 MAE of the model is  0.05677401686009714
 MAPE of the model is  0.01309013836702433


In [104]:
from lazypredict.Supervised import LazyRegressor

In [105]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

 55%|███████████████████████▌                   | 23/42 [03:38<03:11, 10.07s/it]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 76%|████████████████████████████████          | 32/42 [13:04<25:30, 153.03s/it]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 497.


100%|███████████████████████████████████████████| 42/42 [13:43<00:00, 19.61s/it]


In [112]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
QuantileRegressor,10043211367038153645267416946348263767194115537...,-3710801570825834223683526525040227552952848714...,61327106302239143998478578111911239896408497166...,557.45
SGDRegressor,1406113840646634954817536.00,-5195359589619256659017728.00,725648686146.44,0.15
Lars,2356154877172.03,-8705605109546.41,939329.78,0.80
KernelRidge,51.10,-184.09,4.33,0.09
GaussianProcessRegressor,46.50,-167.11,4.13,1.08
LinearRegression,2.38,-4.11,0.72,0.21
TransformedTargetRegressor,2.38,-4.11,0.72,0.18
Lasso,1.27,-0.00,0.32,0.15
LassoLars,1.27,-0.00,0.32,0.12
